# Σκύλοι στην Νέα Υόρκη

Δεδομένα από: [a FOIL request to New York City](https://www.muckrock.com/foi/new-york-city-17/pet-licensing-data-for-new-york-city-23826/)

## Εισάγετε τις βιβλιοθήκες που πρέπει

In [ ]:
import pandas as pd
%matplotlib inline

## Διαβάστε το αρχείο `NYC_Dog_Licenses_Current_as_of_4-28-2016.xlsx` και δείτε τις πρώτες 5 γραμμές.

In [ ]:
df= pd.read_excel("NYC_Dog_Licenses_Current_as_of_4-28-2016.xlsx", nrows = 30000) #na_values = ["???", "Unknown", "UNKNOWN"]
df = df.head(30000)
df.head(5)

## Πόσες γραμμές και πόσες στήλες έχουν τα δεδομένα? Τι τύπος είναι? 

Αν είναι πάνω από 30,000 γραμμές, πηγαίνεται πίσω και διαβάστε μόνο τις πρώτες 30,000.

In [ ]:
df.shape

In [ ]:
df.dtypes

## Ποιες είναι οι πιο δημοφιλείς (primary) ράτσες σκύλων? Κάντε ένα γράφημα με τις top 10.

In [ ]:
df['Primary Breed'].value_counts(ascending = True).tail(10).plot(kind='barh')

## "Unknown" δεν είναι ράτσα. Κάντε ένα γράφημα με τις ράτσες που δεν είναι Unknown

In [ ]:
dogs_not_unknown = df[df['Primary Breed'] != 'Unknown']

dogs_not_unknown['Primary Breed'].value_counts().head().plot(kind='barh')

## Ποια είναι τα πιο δημοφιλή ονόματα σκύλων? 

In [ ]:
df['Animal Name'].value_counts(ascending = True).tail(10).plot(kind='barh') 
#για να μην βγαίνει συνέχεια unknown, όταν εισάγω το αρχείο θέτω τα NANs

## Πόσοι σκύλοι λέγονται  "Max" και πόσοι "Maxwell"?

In [ ]:
#df[df['Secondary Breed'].str.contains("TER", na=False)] #για search μέσα σε string
df[df['Animal Name']=='Max'].shape

In [ ]:
df[df['Animal Name']=='Maxwell'].shape

In [ ]:
df['Animal Third Color'].value_counts().head()

## Τι ποσοστό των σκύλων είναι φύλακες ? (guard dogs)

Δείτε το documentation για το [value counts]για περισσότερα (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html).

In [ ]:
df[df['Guard or Trained']== 'Yes']['Guard or Trained'].value_counts(dropna=False)

In [ ]:
sum_trained = df['Guard or Trained'].value_counts(dropna=False)

In [ ]:
17 / sum_trained.sum() * 100

## Κάτι δεν πάει καλά..που είναι οι υπόλοιποι σκύλοι? Πώς  μπορούμε να τους βρούμε? 

In [ ]:
df['Guard or Trained'].value_counts(dropna=False)

## Γεμίστε τα κενά στην στήλη "Guard or Trained" με "No"

Μετά ξανατσεκάρετε με `.value_counts()`

In [ ]:
df['Guard or Trained'].fillna("No").value_counts()

## Τι ράτσες είναι τα σκυλιά φύλακες? 

In [ ]:
df[df['Guard or Trained']== 'Yes']['Primary Breed'].value_counts()

## Φτιάξτε μια νέα στήλη με το όνομα "year" που να είναι η χρονιά γέννησης του σκύλου. 

Η στήλη `Animal Birth` είναι τύπος datetime, οπότε με τον παρακάτω κώδικα θα γίνει

`df['Animal Birth'].apply(lambda birth: birth.year)`.

In [ ]:
df['year'] = df['Animal Birth'].apply(lambda birth:birth.year)
df.head()

## Υπολογίστε μια νέα στήλη “age” που να δείχνει πόσο χρονών είναι ο σκύλος. Ποιος είναι ο μέσος όρος ηλικίας των σκύλων? 

In [ ]:
df['age'] = 2021 - df['year']
df.head()

In [ ]:
df['age'].mean()

# Ένωση dataframes

## Σε ποια γειτονιά μένει ο κάθε σκύλος? 

Έχουμε ένα αρχείο με τις περιοχές της ΝΥ `zipcodes-neighborhoods.csv`. Ενώστε αυτά τα 2 datasets μαζί. **Προσοχή μην σώσετε το dataframe ως `df`, γιατί θα κάνετε overwrite το dataframe με τους σκύλους.**

In [ ]:
df_zipcodes = pd.read_csv("zipcodes-neighborhoods.csv")
df_zipcodes.head(2)

In [ ]:
combined_df = df.merge(df_zipcodes, left_on="Owner Zip Code", right_on="zip")
combined_df

## Ποιο είναι το πιο συχνό όνομα στο Bronx? στο Brooklyn? στην Upper East Side?

In [ ]:
Bronx_df = combined_df[combined_df['borough']=='Bronx']
Bronx_df
Bronx_df["Animal Name"].value_counts().head(1)

In [ ]:
Brooklyn_df = combined_df[combined_df['borough']=='Brooklyn']
Brooklyn_df
Brooklyn_df["Animal Name"].value_counts().head(1)

In [ ]:
Upper_df = combined_df[combined_df['neighborhood']=='Upper East Side']
Upper_df
Upper_df["Animal Name"].value_counts().head(1)

## Πιο είναι το πιο συχνό όνομα σε όλες τις γειτονιές της ΝΥ μαζί? 

In [ ]:
combined_df.groupby('borough')["Primary Breed"].value_counts().groupby(level=0).head(1)

## Ποια ράτσα σκύλων είναι λιγότερο πιθανό να ψεκαστεί? Θηλυκά ή αρσενικά? 

In [ ]:
breeds = df['Primary Breed'].value_counts(ascending=False)
not_fixed = df[df['Spayed or Neut']== 'No']
not_fixed['Primary Breed'].value_counts(ascending=False)
not_fixed1 = not_fixed['Primary Breed'].value_counts(ascending=False) / df['Primary Breed'].value_counts(ascending=False)
not_fixed1.dropna().sort_values(ascending=False)

In [ ]:
not_fixed["Animal Gender"].value_counts(ascending=False)

In [ ]:
#df[df['Spayed or Neut']== 'No']["Animal Gender"].value_counts()

## Φτιάξτε μια νέα στήλη που να λέγεται monochrome η οποία είναι True για κάθε σκύλο που έχει μαύρο, άσπρο, καφέ ή γκρι ώς ένα από τα χρώματά τους. Πόσα είναι μονόχρωμα? 

In [ ]:
colors = ["BLACK", "BROWN", "GREY", "black", "grey", "white"]
df['Monochrome'] = df['Animal Secondary Color'].isnull() & df['Animal Dominant Color'].isin(colors) 
df['Monochrome'].value_counts()[True]

## Πόσοι σκύλοι υπάρχουν σε κάθε γειτονιά? Φτιάξτε ένα γράφημα. 

In [ ]:
combined_df['borough'].value_counts().groupby(level=0).head(1).plot(kind="barh")

## Φτιάξτε ένα γράφημα με τις top 5 ράτσες σε κάθε borough.

In [ ]:
combined_df.groupby('borough')["Primary Breed"].value_counts().groupby(level=0).head(5).plot(kind='barh')

## Τι ποσοστό των σκύλων δεν είναι φύλακες? 

In [ ]:
df['Guard or Trained'].value_counts(normalize=True)